In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var
from autodiag.perturbation import combine_covs, combine_vars

In [ ]:
import os.path
n_recordings = None#20 # only if I want to limit it
    
folder = 'data/models/pytorch/auto-diag/6000-input/22/'


In [ ]:
from hyperoptim.rerun import rerun_exp
ex = rerun_exp(folder, update_params=dict(only_return_exp=True), 
          save_experiment=False)

In [ ]:
exp = ex.result


In [ ]:
assert hasattr(exp, 'test_dataset')
if n_recordings is not None:
    exp.test_dataset.n_recordings = n_recordings
test_X, test_y = exp.test_dataset.load()


In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget
test_set = SignalAndTarget(test_X, test_y)


In [ ]:
import torch as th
exp.model.load_state_dict(th.load(os.path.join(folder, 'model_params.pkl')))
exp.model.eval();

In [ ]:
setname = 'test'
log.info("Compute and save predictions for {:s}...".format(
                    setname))
dataset = test_set
label_filename = os.path.join(folder, '{:s}_trial_labels.npy'.format(setname))
log.info("Save labels for {:s} to {:s}...".format(
    setname,label_filename))
np.save(label_filename, dataset.y)

In [ ]:
import numpy as np
import torch as th
from torch import nn

from hyperoptim.rerun import rerun_exp
from braindecode.torch_ext.util import var_to_np, np_to_var
from braindecode.datautil.signal_target import SignalAndTarget

In [ ]:
from braindecode.torch_ext.util import var_to_np
from autodiag.monitors import compute_preds_per_trial
preds_per_batch = [var_to_np(exp.model(np_to_var(b[0]).cuda()))
          for b in exp.iterator.get_batches(dataset, shuffle=False)]
preds_per_trial = compute_preds_per_trial(
    preds_per_batch, dataset,
    input_time_length=exp.iterator.input_time_length,
    n_stride=exp.iterator.n_preds_per_input)
mean_preds_per_trial = [np.mean(preds, axis=(0, 2)) for preds in
                            preds_per_trial]
mean_preds_per_trial = np.array(mean_preds_per_trial)

In [ ]:
trial_pred_filename = os.path.join(folder, '{:s}_trial_preds.npy'.format(setname))
log.info("Save trial preds for {:s} to {:s}...".format(
    setname,trial_pred_filename))
np.save(trial_pred_filename, mean_preds_per_trial)

In [ ]:
crop_pred_filename = os.path.join(folder, '{:s}_crop_preds.npy'.format(setname))
log.info("Save crop preds for {:s} to {:s}...".format(
    setname,crop_pred_filename))
np.save(crop_pred_filename, preds_per_trial)